In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset, myNormalize, myToTensor
from torchvision import transforms

In [2]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
mynorm = myNormalize()
my_to_tensor = myToTensor()
transform = transforms.Compose([mynorm, my_to_tensor])

In [4]:
train_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\train\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)
test_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\test\\','D:\python\SoilNet\dataset\LUCAS_2015_Germany_all.csv', transform=transform)

In [5]:
# CONFIG
NUM_WORKERS = 2
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 2

In [31]:
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [32]:
# Get image and label from custom DataLoader
img_custom, label_custom = next(iter(train_dl))

print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label_custom.shape}")

Image shape: torch.Size([8, 12, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([8])


In [33]:
from soilnet.soil_net import SoilNetFC

In [34]:
model = SoilNetFC().to(device)
img_gpu = img_custom.to(device)
print(img_gpu.device)
y = model(img_gpu[1])
y.detach()


cuda:0


tensor([0.1428], device='cuda:0')

In [35]:
torch.min(img_gpu[0]) , torch.min(img_gpu[1])

(tensor(0.0014, device='cuda:0'), tensor(0., device='cuda:0'))

In [51]:
def train_step(model:nn.Module, data_loader:DataLoader, loss_fn:nn.Module, optimizer:torch.optim.Optimizer):
    size = len(data_loader.dataset)
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss = 0
    
    for batch, (X, y) in enumerate(data_loader):
        # Send data to target device
        X, y = X.to(device), y.to(device)
        # 1. Forward pass
        y_pred = model(X)


        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    train_loss = train_loss / len(data_loader)
    return train_loss

In [52]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [53]:
x= train_step(model, train_dl, RMSELoss(), torch.optim.Adam(model.parameters(), lr=1e-4))
x

loss: 0.064519  [    0/   80]
loss: 0.085931  [   16/   80]
loss: 0.048626  [   32/   80]
loss: 0.207757  [   48/   80]
loss: 0.066452  [   64/   80]


0.12356999218463897

In [54]:
# Test step function
def test_step(model:nn.Module, data_loader:DataLoader, loss_fn:nn.Module):
    size = len(data_loader.dataset)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(data_loader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            
            # if batch % 2 == 0:
            #     loss, current = loss.item(), batch * len(X)
            #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    test_loss /= len(data_loader)
    correct /= size
    print(f"Test Loss: {test_loss:>8f}%")
    return test_loss

In [56]:
x= test_step(model, test_dl, RMSELoss())
x

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 0.077496%


0.07749636173248291

In [57]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = RMSELoss(),
          epochs: int = 10):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "test_loss": [],
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss = train_step(model=model,
                                           data_loader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss = test_step(model=model,
            data_loader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | ",
            f"train_loss: {train_loss} | ",
            f"test_loss: {test_loss} | "
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    # 6. Return the filled results at the end of the epochs
    return results

# Model is to Small it can't even overfit! | try RESNET

In [58]:
train(model, train_dl, test_dl, torch.optim.Adam(model.parameters(), lr=1e-4), RMSELoss(), epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.069778  [    0/   80]
loss: 0.210403  [   16/   80]
loss: 0.066395  [   32/   80]
loss: 0.082252  [   48/   80]
loss: 0.149557  [   64/   80]


c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 10%|█         | 1/10 [00:07<01:08,  7.61s/it]

Test Loss: 0.065814%
Epoch: 1 |  train_loss: 0.12189402505755424 |  test_loss: 0.06581447483040392 | 
loss: 0.250558  [    0/   80]
loss: 0.063469  [   16/   80]
loss: 0.052782  [   32/   80]
loss: 0.095896  [   48/   80]
loss: 0.057213  [   64/   80]


 20%|██        | 2/10 [00:15<01:00,  7.56s/it]

...

 90%|█████████ | 9/10 [01:07<00:07,  7.51s/it]

Test Loss: 0.066196%
Epoch: 9 |  train_loss: 0.11500979922711849 |  test_loss: 0.06619635638780892 | 
loss: 0.324800  [    0/   80]
loss: 0.196858  [   16/   80]
loss: 0.125195  [   32/   80]
loss: 0.066769  [   48/   80]
loss: 0.047454  [   64/   80]


100%|██████████| 10/10 [01:18<00:00,  7.88s/it]

Test Loss: 0.067054%
Epoch: 10 |  train_loss: 0.11785470619797707 |  test_loss: 0.06705415789037943 | 


{'train_loss': [0.12189402505755424,
  0.11568898521363735,
  0.11108031123876572,
  0.12073248066008091,
  0.1164714403450489,
  0.11905330382287502,
  0.11887915171682835,
  0.11553867384791375,
  0.11500979922711849,
  0.11785470619797707],
 'test_loss': [0.06581447483040392,
  0.06872266940772534,
  0.06795748928561807,
  0.06474933521822095,
  0.06493690288625657,
  0.06634898874908686,
  0.06889956705272197,
  0.06526545668020844,
  0.06619635638780892,
  0.06705415789037943]}